<a href="https://colab.research.google.com/github/aaolcay/Randomized-Search-CV/blob/main/RandomizedSearchCV_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Hyperparameter Tuning: Randomized Search with Cross-Validation Implementation to CNN model**
`RandomizedSearchCV` is a useful tool in scikit-learn library that allows us to perform hyperparameter tuning by searching through a defined hyperparameter space. It uses random sampling to select combinations of hyperparameters and evaluates them using cross-validation. Here's a step-by-step tutorial on how to use `RandomizedSearchCV`:



**Step 1:** Import the necessary libraries

In [1]:
from keras.datasets import mnist
import tensorflow.keras as keras
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

**Step 2:** Load the dataset

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


**Step 3:** Reshape and normalize the input data

In [3]:
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32')/255.0 # the images have one channel
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32')/255.0

**Step 4:** Build your model 

Here, we built a very simple Convolutional Neural Network (CNN) model to observe the performance and do fine-tuning with `RandomizedSearchCV`.

In [4]:
# Define the CNN model
def create_model(activation_1='relu', activation_2='relu', optimizer='adam', filters_1=16, filters_2=32, kernel_size=3, dense_units=128):
  model = keras.Sequential()
  model.add(keras.layers.Conv2D(filters=filters_1,
                                kernel_size=kernel_size,
                                activation=activation_1,
                                padding="same",
                                input_shape=(28,28,1)))
  model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
  model.add(keras.layers.Conv2D(filters=filters_2,
                                kernel_size=kernel_size,
                                activation=activation_2,
                                padding="same"))
  model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
  # Fully Connected Layer:
  model.add(keras.layers.GlobalAveragePooling2D())
  model.add(keras.layers.Dense(units=dense_units, activation=activation_1))
  model.add(keras.layers.Dense(units=10, activation='softmax'))
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  return model 

**Step 5:** Create the CNN model wrapper for scikit-learn which is required for implementing `RandomizedSearchCV`.

In [ ]:
cnn_model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=1, batch_size=10, verbose=0)

**Step 6:** Define the parameter distributions

Note: The `RandomizedSearchCV` expects the parameter values to be provided as a distribution or an iterable (such as a list or array) for sampling. Since such a function as `random.randint(2, 5)` returns a single integer, it doesn't meet the requirements.

To use a range of values for the 'kernel_size' hyperparameter, we utilize the `scipy.stats.randint` distribution from the `scipy.stats` module (see libraries imported above), as shown below:

In [6]:
param_dist = { 
              'activation_1':['relu', 'sigmoid', 'tanh'],
              'activation_2':['relu', 'sigmoid', 'tanh'],
              'filters_1': randint(8,32),
              'filters_2': randint(16,64),
              'kernel_size': randint(2,5),
              'optimizer': ['adam', 'sgd', 'rmsprop', 'adagrad', 'adadelta'],
              'dense_units': randint(64,256)
              }

**Step 7:** Perform randomized search

In [7]:
random_search = RandomizedSearchCV(
                                   cnn_model, # enter your wrapped model
                                   param_distributions=param_dist, # enter the hyperparameters (you defined in the parameters distributions)
                                   n_iter=20, # how many combinations you want
                                   cv=3 # k-fold cross validation (k=5)
                                   )

In [8]:
random_search.fit(x_train, y_train, verbose=1)

Epoch 1/2
4000/4000 [==============================] - 26s 4ms/step - loss: 2.2923 - accuracy: 0.1301
Epoch 2/2
4000/4000 [==============================] - 16s 4ms/step - loss: 2.2683 - accuracy: 0.1889
Epoch 1/2
4000/4000 [==============================] - 16s 4ms/step - loss: 2.2962 - accuracy: 0.1817
Epoch 2/2
4000/4000 [==============================] - 16s 4ms/step - loss: 2.2820 - accuracy: 0.2076
Epoch 1/2
4000/4000 [==============================] - 16s 4ms/step - loss: 2.2945 - accuracy: 0.1644
Epoch 2/2
4000/4000 [==============================] - 15s 4ms/step - loss: 2.2785 - accuracy: 0.1918
Epoch 1/2
4000/4000 [==============================] - 16s 4ms/step - loss: 2.3050 - accuracy: 0.1069
Epoch 2/2
4000/4000 [==============================] - 15s 4ms/step - loss: 2.2913 - accuracy: 0.1416
Epoch 1/2
4000/4000 [==============================] - 16s 4ms/step - loss: 2.3042 - accuracy: 0.1108
Epoch 2/2
4000/4000 [==============================] - 15s 4ms/step - loss: 2.2726

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f0a6c3a7ca0>,
                   n_iter=20,
                   param_distributions={'activation_1': ['relu', 'sigmoid',
                                                         'tanh'],
                                        'activation_2': ['relu', 'sigmoid',
                                                         'tanh'],
                                        'dense_units': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f0a6c3a7f40>,
                                        'filters_1': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f0a6c3a6c50>,
                                        'filters_2': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f0a6c3a7190>,
                                        'kernel_size': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f0a6c3a7580>,
                                        'optimizer': ['adam', 'sgd', 'rmsprop',
                                                      'adagrad', 'adadelta']})

**Step 8:** Print all hyperparameter results

In [9]:
for i in range(random_search.n_iter):
  score = random_search.cv_results_['mean_test_score'][i] # mean test score as we have cross-validation (scores/k-fold)
  params = random_search.cv_results_['params'][i]
  print(f"{params}\nscore:{score}")

{'activation_1': 'tanh', 'activation_2': 'tanh', 'dense_units': 98, 'filters_1': 10, 'filters_2': 34, 'kernel_size': 2, 'optimizer': 'adagrad'}
score:0.20876666903495789
{'activation_1': 'relu', 'activation_2': 'sigmoid', 'dense_units': 166, 'filters_1': 15, 'filters_2': 52, 'kernel_size': 3, 'optimizer': 'sgd'}
score:0.16034999986489615
{'activation_1': 'tanh', 'activation_2': 'relu', 'dense_units': 204, 'filters_1': 10, 'filters_2': 25, 'kernel_size': 4, 'optimizer': 'rmsprop'}
score:0.9309499859809875
{'activation_1': 'sigmoid', 'activation_2': 'relu', 'dense_units': 244, 'filters_1': 15, 'filters_2': 56, 'kernel_size': 3, 'optimizer': 'adadelta'}
score:0.09653333326180775
{'activation_1': 'sigmoid', 'activation_2': 'sigmoid', 'dense_units': 228, 'filters_1': 11, 'filters_2': 18, 'kernel_size': 2, 'optimizer': 'sgd'}
score:0.10923333217700322
{'activation_1': 'sigmoid', 'activation_2': 'tanh', 'dense_units': 198, 'filters_1': 29, 'filters_2': 60, 'kernel_size': 3, 'optimizer': 'rmsp

**Step 9:** Print optimum hyperparameters with the score

In [10]:
best_score = random_search.best_score_
best_params = random_search.best_params_
print(f"{best_params}\nscore:{best_score}")

{'activation_1': 'tanh', 'activation_2': 'tanh', 'dense_units': 110, 'filters_1': 11, 'filters_2': 52, 'kernel_size': 4, 'optimizer': 'adam'}
score:0.9479666550954183
